# Lightkurve and Periodogram Class

In [11]:
%matplotlib inline
import lightkurve as lk


class KeplerLCPG:
    def __init__(self, target, quarter, oversample_factor=1):
        self.target = target
        self.lc = lk.search_lightcurve(target, author='Kepler', quarter=quarter, cadence='long').download_all().stitch()
        self.pg = self.lc.to_periodogram(oversample_factor=oversample_factor)
        self.period = self.pg.period_at_max_power.value
        
    def plotlc(self):
        return self.lc.plot()

    def plotpg(self, view='period', scale='linear'):
        return self.pg.plot(view=view, scale=scale)
    
    def foldlc(self, period):
        return self.lc.fold(period=period).scatter()
    
    def compare(self, ref):
        return f'{(self.period/ref - 1) * 100:.3f} %'
    
    def __repr__(self):
        return f'{self.target}: {self.period:.3f}'
    
    def __str__(self):
        return self.__repr__()

# Function Comparing to McQuillan 2014

In [25]:
def compare(source, output, n):
    d = open(output, 'w')

    with open(source, 'r') as s:
        line = s.readline()

        for i in range(n):
            line = line.split()

            lcpg = KeplerLCPG(f'KIC {line[0]}', quarter=(4, 5, 6, 7, 8))
            d.write(f'KIC {line[0]:>8}:\t{lcpg.period:<6.3f},\t{line[4]:>6},\t{lcpg.compare(float(line[4])):>10}\n')
            # lcpg.plotpg(scale='log')
            del(lcpg)

            line = s.readline()

    d.close()

def main(n):
    if __name__ == '__main__':
        compare('originaldata.txt', 'comparison.txt', n)

main(n=25)
